In [225]:
#modules
#!pip install requests -q
import itertools
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta

In [266]:
#Create dictionaries of search parameters
titleDict = {
    0: 'Intern', 1: 'Data', 2: 'BI', 3: 'Developer', 4: 'ML', 5: 'Machine-Learning', 6: 'Applied', 7: 'Modeling', 
    8: 'Quantitative', 9: 'Analyst',  10: 'Decision', 11: 'Scientist', 12: 'Business',  13: 'Intelligence', 
    14: 'Artificial', 15: 'AI', 16: 'Junior', 17: 'Jr', 18: 'Engineer', 19: 'Big', 20: 'Software'
}

locationDict = {
    0: 'Pittsburgh',  1: 'Nashville',  2: 'Charlotte',  3: 'California',  4: 'Austin',  5: 'Minneapolis',
    6: 'San-Antonio',  7: 'Massachusetts'
}

skills = [
    'Python', 'PySpark', 'Excel', 'EVIEWS', 'STATA', '.NET', 'Agile', 'Git', ''
]

In [264]:
#class for each job posting
class job_posting:
    #attributes
    
    def __init__(self): pass
    
    def __init__(self, title, company, location, date, skills, visa, relo):
        self.job_title = title
        self.company_name = company
        self.job_location = location
        self.posted_date = date
        self.tech_skills = get_skills(skills)
        self.immigration_assistance = visa
        self.relocation_assistance = relo
        
    def get_skills_requested(self, description):
        skills = []
        #parse through description to extract skills if they are in your dictionary and return list
        return skills

In [228]:
#job name combinations from dictionary in search results
def get_job_titles(i):
    values = list(itertools.product(titleDict.values(), repeat=i))
    titles = []
    for words in values: 
        dups = [words.count(x) for x in list(words)] #duplicates in set of three
        if sum(dups) == i and sum(list(map(len, list(words)))) > i*2  and set(words) not in titles:
            titles.append(set(words))
    titles = ['-'.join(list(words)) for words in titles]
    print(i, 'values combination', 'all combinations:', len(values), 'filtered values:', len(titles))
    return titles

In [244]:
titles = get_job_titles(2) + get_job_titles(3)

2 values combination all combinations: 441 filtered values: 204
3 values combination all combinations: 9261 filtered values: 1326


In [230]:
#method to get search results for positions and locations in the dictionaries defined
#Posted in the last 24 hours and <= 10 miles from job location
def get_results_by_location(title, location):
    links = []
    URL = 'https://www.linkedin.com/jobs/search?keywords='+ title +'&location='+ location +'&f_TP=1&distance=10'
    page = requests.get(URL)
    soup = bs(page.content, 'lxml')
    refs = soup.find_all('a', class_='result-card__full-card-link')
    links = [ref.get('href') for ref in refs if ref.get('href') not in links]
    return links

In [260]:
#method to get all results for desired locations
def get_all_location_results(job_titles, location): 
    start = 0
    cut_off = 20
    end = len(job_titles)
    links = []
    while start <= end and cut_off <= end: 
        for title in job_titles[start: cut_off]:
            links += get_results_by_location(title, location)    
            start = cut_off + 1
            cut_off += 20
            if cut_off > end:
                for title in job_titles[start: end]:
                    links += get_results_by_location(title, location)    
                break
    print(location+':', len(links), 'results')
    return links

In [262]:
def get_posting_links():    
    stime = datetime.now()
    links = []
    print(len(titles), 'search terms: \n-----------------------------\n')
    for location in locationDict.values():
        links += get_all_location_results(titles, location)  
    etime = datetime.now()
    mins = (etime-stime)
    print('\nAll results:', len(links))
    print('total time:', round(mins.seconds/60), 'mins:' + str(minutes.seconds%60), 'secs')

In [263]:
links[109]

'https://www.linkedin.com/jobs/view/atg-developer-at-caspex-1831583854?refId=c1da6d94-3cea-49ce-9110-906cc47e3c55&position=13&pageNum=0&trk=public_jobs_job-result-card_result-card_full-click'

In [ ]:
for link in links:
    page = requests.get(link)
    soup = bs(page.content, 'lxml')